In [2]:
import os
from PIL import Image
import json

For each threefold foggy image calculate the corresponding SAM map number and save in a dictionary

In [21]:
sets = ['train', 'val']
for s in sets:
    conversions = dict()
    fog_path = os.path.join('/Users/nathanieljames/Desktop/dl_final_proj/cityscape_no_val/fog/leftImg8bit_foggyDBF', s)
    count = 1
    level = 0
    for city in sorted(os.listdir(fog_path)):
        #continue if the city is not a directory
        city_path = os.path.join(fog_path, city)
        if not os.path.isdir(city_path):
            continue
        for img in sorted(os.listdir(city_path)):
            img_path = os.path.join(city_path, img)
            conversions[img_path] = count
            level += 1
            if level == 3:
                count += 1
                level = 0
    save_path = os.path.join('/Users/nathanieljames/Desktop/dl_final_proj/conversion_dicts', f'{s}.json')
    print(save_path)
    with open(save_path, "w") as f:
        json.dump(conversions, f)

/Users/nathanieljames/Desktop/dl_final_proj/conversion_dicts/train.json
/Users/nathanieljames/Desktop/dl_final_proj/conversion_dicts/val.json


Load the dictionaries, and combine foggy image with SAM map

In [7]:
sets = ['train', 'val']
for s in sets:
    dict_path = os.path.join('/Users/nathanieljames/Desktop/dl_final_proj/conversion_dicts', f'{s}.json')
    sam_base = '/Users/nathanieljames/Desktop/dl_final_proj/cityscapes_data'
    write_base = '/Users/nathanieljames/Desktop/dl_final_proj/target_fog'
    comb_path = os.path.join(write_base, 'combined', s)
    with open(dict_path, "r") as f:
        conv_dict = json.load(f)
    for fog_path in conv_dict:
        level = int(fog_path.split('.')[1])
        level_path = os.path.join(write_base, f'{level}', s)
        sam_target = conv_dict[fog_path]
        sam_path = os.path.join(sam_base, f'{s}', f'{sam_target}.jpg')
        img1 = Image.open(fog_path)
        img2 = Image.open(sam_path)
        img1 = img1.resize((256, 256))
        # concatenate images
        dst = Image.new('RGB', (512, 256))
        dst.paste(img2, (0, 0))
        dst.paste(img1, (0, 0))
        wp1 = os.path.join(comb_path, f'{sam_target}-{level}.jpg')
        wp2 = os.path.join(level_path, f'{sam_target}.jpg')
        dst.save(wp1)
        dst.save(wp2)

Rename combined dataset to allow indexing

In [9]:
sets = ['train', 'val']
write_base = '/Users/nathanieljames/Desktop/dl_final_proj/target_fog'
for s in sets:
    comb_path = os.path.join(write_base, 'combined', s)
    tmp_path = os.path.join(comb_path, 'temp')
    count = 1
    for img in os.listdir(comb_path):
        #move image to tmp folder and rename it to {count}.png
        if img == 'temp':
            continue
        img_path = os.path.join(comb_path, img)
        tmp_img_path = os.path.join(tmp_path, f'{count}.png')
        os.rename(img_path, tmp_img_path)
        count += 1

In [10]:
sets = ['train', 'val']
for s in sets:
    comb_path = os.path.join(write_base, 'combined', s)
    tmp_path = os.path.join(comb_path, 'temp')
    for img in os.listdir(tmp_path):
        img_path = os.path.join(tmp_path, img)
        os.rename(img_path, os.path.join(comb_path, img))

In [23]:
sets = ['val']
write_base = '/Users/nathanieljames/Desktop/dl_final_proj/target_fog'
for s in sets:
    comb_path = os.path.join(write_base, '5', s)
    for i in range(1, 8927):
        img_path = os.path.join(comb_path, f'{i}.jpg')
        if i == 1339:
            print(os.path.exists(img_path))
        if not os.path.exists(img_path):
            print(img_path)
            print(i)
            break


/Users/nathanieljames/Desktop/dl_final_proj/target_fog/5/val/501.jpg
501
